In [1]:
import numpy as np
np.random.seed(2671)  # for reproducibility

import copy
import json
import cv2
from scipy.misc import imread

from keras.models import Sequential, load_model
from keras import backend as K
import matplotlib.pyplot as plt
import sys

import pickle

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 70.0% of memory, cuDNN 5103)
/Library/Python/2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
photo_ids = []
input_photos = []
with open("small_dog_data.json") as data_file:    
    dog_data = json.load(data_file)
for k,v in dog_data.iteritems():
    nBreeds = float(len(v["breeds"]))
    if len(v["breeds"]) > 1: continue
    breed = v["breeds"][0]
    photos = v["photos"]
    for photo in photos:
        imagename = "cutPetFinder/"+k+"_"+photo["id"]+".jpg"
        img = cv2.imread(imagename)
        if img is None: continue
        img = cv2.resize(img, (299,299), cv2.INTER_LINEAR)
        photo_ids.append(k+"_"+photo["id"])
        input_photos.append(img)
        #break

In [3]:
print len(input_photos)
batch_size = 10
nb_epoch = 5

train = np.array(input_photos)

print train.shape

nToUse = len(train)
testFraction = 1.0/3.0
nTest = int(testFraction*nToUse)

X_test = train[:]

# input image dimensions
img_rows, img_cols = X_test.shape[1], X_test.shape[2]

if K.image_dim_ordering() == 'th':
    X_test = np.swapaxes(X_test,1,3)
    X_test = np.swapaxes(X_test,2,3)
    input_shape = (X_test.shape[1], img_rows, img_cols)
else:
	input_shape = (img_rows, img_cols, X_test.shape[3])

print input_shape

X_test = X_test.astype('float32')
X_test /= 255

3301
(3301, 299, 299, 3)
(3, 299, 299)


In [8]:
model = load_model("dog_cnn_inception.1.10.keras")
#model = load_model("dog_cnn_simple.keras")

In [9]:
results = model.predict(X_test)

In [10]:
output = dict(zip(photo_ids,results))

In [11]:
with open('petfinderPurebred_dog_cnn_inception_all.pickle', 'wb') as handle:
    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)
#with open('petfinderPurebred_dog_cnn_simple_all.pickle', 'wb') as handle:
#    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
print results.shape

(3301, 120)
